# Data Cleaning

Bevor du mit der Textanalyse beginnst, musst du die Daten in eine geeignete Form bringen. Dazu gehört leider auch das lästige *Aufräumen* der Daten.

Die Datensätze kannst du stichprobenartig überprüfen. Wenn du dabei Ungereimtheiten feststellst, musst du ein Schema finden, mit denen du diese beseitigen kannst.

Ein paar der häufigsten Probleme schauen wir uns in diesem Notebook an.

## Zeichensätze im Heise Newsticker

Auf der Erde gibt es sehr viel Sprachen und einige haben auch ihre eigenen Zeichen. In der Anfangszeit der Computer wurde ein 7-bit-Zeichensatz ([ASCII](https://de.wikipedia.org/wiki/American_Standard_Code_for_Information_Interchange)) festgelegt, der für die englische Sprache aus ausgereichet hat.

Für europäische Sprachen hat das aber nicht genügt, daher gab es viele Erweiterungen auf 8-bit, von denen besonders [ISO-8859](https://de.wikipedia.org/wiki/ISO_8859) wichtig ist, weil diese noch immer zum Einsatz kommt.

Aber auch acht Bit haben langfristig nicht genügt, deswegen wurde gleich ein 32-Bit-Zeichensatz definiert, der sog. [Unicode](https://de.wikipedia.org/wiki/Unicode). Der Zeichenvorrat darin wird langfristig genügen.

Unicode selbst hat wieder unterschiedliche Kodierungen, von denen heute aus verschiedenen Gründen primär [UTF-8](https://de.wikipedia.org/wiki/UTF-8) relevant ist. Es sollte also unser Ziel sein, alles in UTF-8 abszuspeichern bzw. Text in dieses Format zu wandeln.

In Webseiten steht normalerweise die Kodierung im HTML-Header (oder sogar im HTTP-Header). Beim Heise-Newsticker-Artikel kannst du das überprüfen. Da du dazu auch die HTTP-Header-Informationen benötigst, musstdu die Seite nochmal herunterladen:

In [ ]:
import requests
r = requests.get('https://www.heise.de/news/Guten-Rutsch-und-ein-gesundes-neues-Jahr-2021-5001311.html')

Du interessierst dich für den `Content-Type`: 

In [ ]:
r.headers["Content-Type"]

Das sieht also schon sehr gut aus, eine weitere Transformation ist nicht mehr notwendig. Oft steht das *Encoding* auch nochmal im HTML als `<meta charset>`:

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content)
soup.meta.get("charset")

Nicht immer muss das so einfach funktionieren, ab und zu ist das etwas versteckt. Eine gute Übersicht findest du unter https://stackoverflow.com/questions/18359007/trying-to-get-encoding-from-a-webpage-python-and-beautifulsoup

## Zeichensätze erraten und transformieren

Die Idee der Erklärung hier findest du auch unter https://www.tutorialspoint.com/beautiful_soup/beautiful_soup_encoding.htm

Wir konstruieren nun ein Dokument, das ein "falsches" Encoding hat. Dazu müssen wir in Python das *binary encoding* verwenden und geben 8-bit-Zeichen in *griechisch* ein:

In [ ]:
greek = b"<h1>\xed\xe5\xec\xf9</h1>"
soup = BeautifulSoup(greek)
soup.h1

Das originale Encoding lässt sich auch feststellen:

In [ ]:
soup.original_encoding

`BeautifulSoup` hat diese Konvertierung für dich also bereits durchgeführt. Dazu hat `BeautifulSoup` aber "gerraten". Du kannst ein Encoding auch erzwingen:

In [ ]:
soup = BeautifulSoup(greek, from_encoding="iso-8859-15")
soup.h1

## Strings konvertieren

Solltest du den Content nicht als HTML haben, kannst du die Konvertierung natürlich auch direkt in Python durchführen. Dein griechischer Text von oben ist ja kein echtes 

In [ ]:
greek.decode("utf-8", "strict")

In [ ]:
greek.decode("iso-8859-7", "strict")

## Fußzeilen entfernen

Oft sind besonders in extrahierten PDF-Dokumenten Teile enthalten, die du nicht haben möchtest. In dem PDF von GAIA-X findest du z.B. Seitenzahlen und Copyright-Informationen:

In [ ]:
text = """Figure 7:  Schematic inheritance relations and properties for the top-level Self-Description  
Schemas.

© BMWi

132  CORE ARCHITECTURE ELEMENTS

2.5 Catalogue

The concept Self-Description is the foundation of the 
federated GAIA-X Catalogues. Catalogues are the 
main building block for the publication and discovery 
of Self-Descriptions of Assets and Participants. To sat-
isfy Consumer needs and to objectively find the best 
fitting offerings in the tangle of registered Assets, an 
open and transparent query algorithm is implemented 
without any GAIA-X internal ranking. Beside search 
functionality, a graph-based navigation interface is 
provided to traverse the complex tangle of offered 
Services, Nodes and linked Self-Descriptions, includ-
ing the attached claims with chain of trust statements. 
Consumers can verify each Self-Description individu-
ally and decide which one to select in a self-sovereign 
manner – GAIA-X does not act as a runtime interme-
diary or broker.
"""

Um diese entfernen zu können, arbeitest du am besten mit [*Regular Expressions*](https://de.wikipedia.org/wiki/Regul%C3%A4rer_Ausdruck). Python bietet dir dafür eine (leider nicht besonders gut integrierte) Bibliothek an, mit der du Ersetzungen durchführen kannst. Du löschst alle Zeilen, die nur mit Ziffern beginnen `[0-9]+` und anschließend ein Trennzeichen enthalten:

In [ ]:
!pip install regex

In [ ]:
# regex ist deutlich leistungsfähiger als re, aber kompatibel
import regex as re
text = re.sub(r'^[0-9]+\s.*', '', text, flags=re.MULTILINE)
print(text)

Das `© BMWi` kannst du auch ganz leicht entfernen:

In [ ]:
text = re.sub(r'^© BMWi.*', '', text, flags=re.MULTILINE)
print(text)

## Trennungen wiederherstellen

Das sieht eigentlich schon ganz gut aus, jetzt müssen noch die Trennungen wieder zu ganzen Wörtern zusammengefügt werden. Der reguläre Ausdruck ist deutlich komplizierter. Über die Klammern wählst du zuerst Wörter aus, die am Ende der Zeile mit `-` enden, dann einen Umbruch haben, mit Kleinbuchstaben weitergehen und mit einem Trenner enden. Diese werden dann über die *Backreferences* (die den Inhalt der runden Klammern verwenden) wieder zusammengefügt:

In [ ]:
text = re.sub(r'(\s[\w]+)\-\n(^[[:lower:]]\w*\s)', '\g<1>\g<2>', text, flags=re.MULTILINE)
print(text)

## Absätze zusammenführen

Ab und zu wird es dir passieren, dass deine Dokumente zu lang sind und du lieber einzelne Absätze auswerten möchtest. Oft funktioniert das ganz gut, aber in dem obigen Dokument sind Zeilenumbrüche aus Layout-Gründen eingefügt, die gar keine Absätze voneinander trennen.

Diese kannst du einfach mit `text.split("\n\n")` zusammenführen. Um sicherzustellen, dass nicht unsichtbare Leerzeichen am Ende einer Zeile (oder in einer leeren Zeile stehen), ist die Lösung mit `re` etwas stabiler:

In [ ]:
re.split('\s*\n\s*\n', text)

## Tippfehler beseitigen

Häufig wirst du Dokumenten begegnen, die Tippfehler enthalten. Damit ist nicht ganz einfach umzugehen, besonders bei nutzergeneriertem Content. Eine automatische Rechtschreibkorrektur (z.B. mit `aspell`) kann helfen, allerdings wird manchmal auch domnänenspezifisches Vokabular verwendet, das dadurch verfälscht wird.

## Dokumente in Form bringen

Das PDF-Dokument hat nun eine bessere Form für die Auswertung bekommen, ist aber sicher nicht perfekt.

Du solltest keinesfalls die Zeit für das *Cleaning* unterschätzen und auf jeden Fall mit einplanen, dass du auch zu späteren Zeitpunkten in der Verarbeitung noch Probleme im Cleaning erkennst und diesen Schritt dann nochmal ausführen musst.